generate TEXT lên hình với yêu cầu là kết quả trả về chứa hình đã generate text + bounding box của text

In [2]:
#pip install pillow

In [108]:
from PIL import Image, ImageDraw, ImageFont

In [110]:
# Tạo nền theo kích thước được cho
width, height = 800, 400
image_bg = Image.new('RGB', (width, height), color='white')

# Tạo đối tượng ImageDraw
draw = ImageDraw.Draw(image_bg)

# Chọn font chữ và kích thước kèm văn bản muốn chèn vào hình
font_size = 50
font = ImageFont.truetype("arial.ttf", font_size)
text = "Cty DIGI-TEXX"

# Tính toán bounding box của văn bản
bbox = draw.textbbox((0, 0), text, font=font)
text_width = bbox[2] - bbox[0]
text_height = bbox[3] - bbox[1]

# Tính toán vị trí để văn bản nằm giữa hình ảnh
image_width, image_height = image_bg.size
x = (image_width - text_width) // 2
y = (image_height - text_height) // 2

# Vẽ văn bản vào hình ảnh
draw.text((x, y), text, font=font, fill=(0, 0, 0))

# Hiển thị hình ảnh
# image_bg.show()
image_bg.save("image_with_text.png")

In [10]:
#pip install --upgrade pip

  Using cached pip-24.2-py3-none-any.whl.metadata (3.6 kB)
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.8 MB 217.9 kB/s eta 0:00:09
   - -------------------------------------- 0.1/1.8 MB 544.7 kB/s eta 0:00:04
   --- ------------------------------------ 0.1/1.8 MB 944.1 kB/s eta 0:00:02
   ------ --------------------------------- 0.3/1.8 MB 1.6 MB/s eta 0:00:01
   ----------- ---------------------------- 0.5/1.8 MB 2.1 MB/s eta 0:00:01
   ----------- ---------------------------- 0.5/1.8 MB 2.1 MB/s eta 0:00:01
   ------------------ --------------------- 0.8/1.8 MB 2.6 MB/s eta 0:00:01
   ----------------------- ---------------- 1.0/1.8 MB 2.8 MB/s eta 0:00:01
   ----------------------------- ---------- 1.3/1.8 MB 3.1 MB/s eta 0:00:01
   ----------------------------- ---------- 1.3/1.8 MB 3.1 MB/s eta 0:00:01
   ----------------------------- ----

In [12]:
#pip install opencv-python

  Using cached opencv_python-4.10.0.84-cp37-abi3-win_amd64.whl.metadata (20 kB)
Using cached opencv_python-4.10.0.84-cp37-abi3-win_amd64.whl (38.8 MB)
Note: you may need to restart the kernel to use updated packages.


In [4]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [14]:
# Giữ lại đoạn này 
# Đọc ảnh gốc
img = cv2.imread('image_with_text.png')

# Chuyển sang ảnh xám
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Inverse ảnh để các vùng màu đen trở thành trắng (và ngược lại)
inverted_img = cv2.bitwise_not(gray)

# Đặt kernel có kích thước lớn hơn (10x10)
kernel = np.ones((15, 15), np.uint8)

# Áp dụng phép dilation lên ảnh đảo ngược
img_dilation = cv2.dilate(inverted_img, kernel, iterations=2)

# Đảo ngược lại ảnh để có dilation trên vùng đen ban đầu
img_dilation_inverted = cv2.bitwise_not(img_dilation)

# Hiển thị kết quả
#cv2.imshow('Dilation on Black Areas', img_dilation_inverted)
#cv2.waitKey(0)
#cv2.destroyAllWindows()
img_dilation

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [20]:
# Tìm chỉ số của các phần tử có giá trị 1
indices = np.argwhere(img_dilation == 1)

# Nếu có ít nhất một phần tử có giá trị 1
if indices.size > 0:
    # Tìm tọa độ có giá trị x và y nhỏ nhất
    min_index = indices[np.argmin(indices[:, 0])]  # Tìm phần tử có x nhỏ nhất
    min_x, min_y = min_index

print(min_x)
print(min_y)

185
218


In [30]:
img_dilation.shape

(400, 800)

In [40]:
# Tìm chỉ số của các phần tử có giá trị 1
indices = np.argwhere(img_dilation == 1)

# Nếu có ít nhất một phần tử có giá trị 1
if indices.size > 0:
    # Tìm tọa độ có giá trị x và y lớn nhất
    max_index = indices[np.argmax(indices[:, 0])]  # Tìm phần tử có x lớn nhất
    max_x, max_y = max_index

    print(f"Tọa độ x lớn nhất: {max_x}")
    print(f"Tọa độ y lớn nhất: {max_y}")
else:
    print("Không có phần tử nào có giá trị 1 trong mảng.")

Tọa độ x lớn nhất: 185
Tọa độ y lớn nhất: 218


In [259]:
# Tìm các đường viền
contours = cv2.findContours(img_dilation_inverted, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2]

# Khởi tạo các giá trị min
min_x, min_y = np.inf, np.inf

# Duyệt qua từng contour
for contour in contours:
    # Tính toán hình chữ nhật bao quanh
    x, y, w, h = cv2.boundingRect(contour)
    
    # Cập nhật các giá trị min
    if x < min_x:
        min_x = x
    if y < min_y:
        min_y = y

# In ra tọa độ thấp nhất
print(f"Lowest x: {min_x}")
print(f"Lowest y: {min_y}")

# Vẽ điểm thấp nhất lên ảnh (tùy chọn)
cv2.circle(img, (min_x, min_y), 10, (0, 255, 0), -1)

# Hiển thị kết quả
cv2.imshow('Image with Lowest Point', img)
cv2.waitKey(0)
cv2.destroyAllWindows()


Lowest x: 0
Lowest y: 0


sửa ảnh đã dilation tìm countour khoanh vùng

In [248]:
img = cv2.imread('image_with_text.png')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
inverted_img = cv2.bitwise_not(gray)
kernel = np.ones((5, 5), np.uint8)
img_dilation = cv2.dilate(inverted_img, kernel, iterations=3)
img_dilation_inverted = cv2.bitwise_not(img_dilation)

ret, thresh = cv2.threshold(img_dilation_inverted, 120, 255, cv2.THRESH_BINARY)
contours = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)[-2]

for contour in contours:
    # Vẽ đường viền
    cv2.drawContours(img, contour, -1, (0, 255, 0), 3)
    
    # Tính toán bounding box quanh đường viền
    x, y, w, h = cv2.boundingRect(contour)
    
    # Vẽ bounding box bằng một hình chữ nhật
    cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)  # Màu xanh dương (BGR), độ dày 2 pixel
    

cv2.imwrite('image_with_text_countour_bounding.png', img)

# cv2.imshow('Dilation on Black Areas', img_dilation_inverted)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

True

In [129]:
import numpy as np
import cv2

img = cv2.imread('image_with_text.png')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(gray, 120, 255, cv2.THRESH_BINARY)
contours = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)[-2]
for contour in contours:
    cv2.drawContours(img, contour, -1, (0, 255, 0), 3)

# Tính toán và in ra giá trị min và max của các contour
all_points = np.concatenate([c.reshape(-1, 2) for c in contours], axis=0)
print(all_points)
min_val = np.min(all_points, axis=0)
max_val = np.max(all_points, axis=0)

# Vẽ hình chữ nhật từ tọa độ min và max
top_left = tuple(min_val)
bottom_right = tuple(max_val)
#cv2.rectangle(img, top_left, bottom_right, (0, 0, 255), 2)

print(f"Min value: {min_val}")
print(f"Max value: {max_val}")

cv2.imwrite('rick-morty-contour.png', img)

[[  0   0]
 [  0 399]
 [799 399]
 ...
 [240 188]
 [241 188]
 [242 187]]
Min value: [0 0]
Max value: [799 399]


True

In [220]:
import numpy as np
import cv2

img = cv2.imread('image_with_text.png')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(gray, 120, 255, cv2.THRESH_BINARY)
contours = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)[-2]
for contour in contours:
    cv2.drawContours(img, contour, -1, (0, 255, 0), 3)
x2,y2,w,h = cv2.boundingRect(contour)
x1 = x2-w
y1 = y2-h
print(x1)
print(x2)
print(y1)
print(y2)
cv2.imwrite('image_with_text_countour_bounding.png', img)

199
232
148
186


True

In [233]:
img = cv2.imread('image_with_text.png')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
inverted_img = cv2.bitwise_not(gray)
kernel = np.ones((5, 5), np.uint8)
img_dilation = cv2.dilate(inverted_img, kernel, iterations=3)
img_dilation_inverted = cv2.bitwise_not(img_dilation)

ret, thresh = cv2.threshold(img_dilation_inverted, 120, 255, cv2.THRESH_BINARY)
contours = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)[-2]

for contour in contours:
    # Vẽ đường viền
    cv2.drawContours(img, contour, -1, (0, 255, 0), 3)
    
    # Tính toán bounding box quanh đường viền
    x2,y2,w,h = cv2.boundingRect(contour)
    start_point = (x2-w, y2-h)
    end_point = (x2, y2)
    print(x2)
    print(y2)
    
    # Vẽ bounding box bằng một hình chữ nhật
    cv2.rectangle(img, (x1, y1), (x1 + w, y1 + h), (255, 0, 0), 2)  # Màu xanh dương (BGR), độ dày 2 pixel
    

cv2.imwrite('image_with_text_countour_bounding.png', img)

# cv2.imshow('Dilation on Black Areas', img_dilation_inverted)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

0
0
433
180
485
212
504
201
533
200
316
180
389
198
334
198
226
180
244
199


True

In [198]:
import numpy as np
import cv2 as cv
 
img = cv.imread('image_with_text.png', cv.IMREAD_GRAYSCALE)
ret,thresh = cv.threshold(img,127,255,0)
contours,hierarchy = cv.findContours(thresh, 1, 2)
 
cnt = contours[2]
M = cv.moments(cnt)
print( M )

{'m00': 451.0, 'm10': 151764.5, 'm01': 92229.5, 'm20': 51081243.666666664, 'm11': 31035840.25, 'm02': 18885186.333333332, 'm30': 17196910467.850002, 'm21': 10446114329.833334, 'm12': 6354969309.533333, 'm03': 3871940320.75, 'mu20': 11479.896711014211, 'mu11': 0.0, 'mu02': 24253.58333333209, 'mu30': 6064.2610206604, 'mu21': 8.530914783477783e-07, 'mu12': -21513.715373335406, 'mu03': 9.5367431640625e-07, 'nu20': 0.056439726014199595, 'nu11': 0.0, 'nu02': 0.11924023644589797, 'nu30': 0.0014039011553114201, 'nu21': 1.974941559999586e-13, 'nu12': -0.00498051283821182, 'nu03': 2.207794931702594e-13}


In [200]:
area = cv.contourArea(cnt)

In [202]:
print(area)

451.0


In [204]:
perimeter = cv.arcLength(cnt,True)
print(perimeter)

84.97056245803833


In [208]:
x,y,w,h = cv.boundingRect(cnt)
print(x)
print(y)
print(w)
print(h)
cv.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
cv.imwrite('image_with_text_countour_bounding.png', img)

328
191
20
28


True